#### Imports

In [16]:
import pandas as pd
import gzip
import contractions
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import d2l
from textblob import TextBlob
from nltk.corpus import stopwords
nltk.download("stopwords")
nltk.download('vader_lexicon')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/enriquelopez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/enriquelopez/nltk_data...


True

## Load Data

Data stored from various categories. Categories include (

Method to load data taken from : 

In [ ]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

## Clean Data

## Useful Code Snipits

##### Group by asin which is tthe product ID

In [1]:
df_group = df.groupby("asin")
group = df_group.get_group("120401325X")["reviewText"].tolist()
group

NameError: name 'df' is not defined

## Feature : The star rating

The review-assigned product star rating

#### EDA

## Feature : Subjectivity 

The probability of a review and its sentences being subjective. https://textblob.readthedocs.io/en/dev/

nltk.sentiment.util.demo_sent_subjectivity

#### EDA

## Feature : TF-IDF  |  Product features

Attempt to identify the presence of important topics

In [14]:
def tf_idf(document):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(document)
    return X.toarray()

stop_words = set(stopwords.words('english'))

lemmatizer = WordNetLemmatizer()

# A Tokenizer that tokenize a sequence of alphabetical character and drop everything else
tokenizer = RegexpTokenizer(r'\w+') 

def get_aspects(x):
    import contractions
    doc = contractions.fix(x)
    doc=tokenizer.tokenize(doc) ## Tokenize and extract grammatical components
    doc=list(map(lambda i: i.lower(),doc)) ## Normalize text to lower case
    doc=[i for i in doc if i not in stop_words] ## Remove common words and retain words not in stop words
    doc = [lemmatizer.lemmatize(j) for j in doc]
    doc=pd.Series(doc)
    doc=doc.value_counts().head(10).index.tolist() ## Get 10 most frequent words
    return doc

get_aspects(df_group.get_group("120401325X")["reviewText"][5])

NameError: name 'df_group' is not defined

#### EDA

## Feature : Sensitivity Analysis

Looking for use of pos/neg words

In [17]:
sid = SentimentIntensityAnalyzer()
test_subset = tokenizer.tokenize(df_group.get_group("120401325X")["reviewText"][3])
pos_word_list=[]
neu_word_list=[]
neg_word_list=[]

for word in test_subset:
    if (sid.polarity_scores(word)['compound']) >= 0.5:
        pos_word_list.append(word)
    elif (sid.polarity_scores(word)['compound']) <= -0.5:
        neg_word_list.append(word)
    else:
        neu_word_list.append(word)                

print('Positive :',pos_word_list)        
print('Neutral :',neu_word_list)    
print('Negative :',neg_word_list)

NameError: name 'df_group' is not defined

#### EDA

## Feature : Readability 

In [ ]:
def readabilityMetrics(text):
    response = requests.post("https://ipeirotis-readability-metrics.p.rapidapi.com/getReadabilityMetrics",
      headers={
        "X-RapidAPI-Host": "ipeirotis-readability-metrics.p.rapidapi.com",
        "X-RapidAPI-Key": "818e0f0b58mshe4a4d5842f257bep1cc5a6jsne73e02710bab",
        "Content-Type": "application/x-www-form-urlencoded"
      },
      params={
        "text":text                        
      }
    )
    rowEntry = response.json()
    rowEntry["TEXT"] = text
    return json_normalize(rowEntry)

#### EDA

## Feature : Spelling Errors

In [ ]:
def numberOfSpellingErrors(text):
    response = requests.get("https://montanaflynn-spellcheck.p.rapidapi.com/check/",
      headers={
            "X-RapidAPI-Host": "montanaflynn-spellcheck.p.rapidapi.com",
            "X-RapidAPI-Key": "818e0f0b58mshe4a4d5842f257bep1cc5a6jsne73e02710bab"
      },
      params={
        "text":text                        
      }
    )
    return len(response.json()['corrections'])

#### EDA

## Define the Model

Since we are predicting a value between [0.0, 5.0] we will be using a linear regression model. 

In [ ]:
def get_net(ctx):

## Define the Training Functions

## Classify the Testing Set

## Measure Accuracy

## Questions at Check-In

1. How can we make our project more novel
2.
3.
4.
5.
6.